In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import time

### **Data Reading**

In [0]:
df=spark.read.format("parquet")\
    .load("abfss://bronze@databricksaniete.dfs.core.windows.net/customers")
df.limit(10).display()
df.count()

customer_id,first_name,last_name,email,city,state,_rescued_data
C00001,Emily,Mooney,rushjeff@ryan.org,Johnsonmouth,MS,null
C00002,Andrea,Sellers,mccoykiara@kelly.com,Stephenfort,WY,null
C00003,Craig,Hayes,rebeccamiller@yahoo.com,South Stephenshire,LA,null
C00004,Bryan,Scott,lawrence05@campbell.info,Chrisland,ND,null
C00005,Sean,Vasquez,carrie45@yahoo.com,East Dennistown,RI,null
C00006,Kevin,Mccarthy,traceyramos@gmail.com,North Matthew,IN,null
C00007,Amanda,Doyle,scottallen@gmail.com,Joneshaven,VA,null
C00008,Paul,Campos,sullivanjeremy@horton-adams.com,South Nathanfurt,CT,null
C00009,Mary,Green,dennis03@yahoo.com,Kimberlyview,MD,null
C00010,James,Myers,charles58@murillo.net,West Hector,OK,null


2000

In [0]:
df=df.drop("_rescued_data")
df.limit(10).display()
df.count()

customer_id,first_name,last_name,email,city,state
C00001,Emily,Mooney,rushjeff@ryan.org,Johnsonmouth,MS
C00002,Andrea,Sellers,mccoykiara@kelly.com,Stephenfort,WY
C00003,Craig,Hayes,rebeccamiller@yahoo.com,South Stephenshire,LA
C00004,Bryan,Scott,lawrence05@campbell.info,Chrisland,ND
C00005,Sean,Vasquez,carrie45@yahoo.com,East Dennistown,RI
C00006,Kevin,Mccarthy,traceyramos@gmail.com,North Matthew,IN
C00007,Amanda,Doyle,scottallen@gmail.com,Joneshaven,VA
C00008,Paul,Campos,sullivanjeremy@horton-adams.com,South Nathanfurt,CT
C00009,Mary,Green,dennis03@yahoo.com,Kimberlyview,MD
C00010,James,Myers,charles58@murillo.net,West Hector,OK


2000

In [0]:
df=df.withColumn('domains',split(col('email'),"@")[1])
df.limit(10).display()
df.count()

customer_id,first_name,last_name,email,city,state,domains
C00001,Emily,Mooney,rushjeff@ryan.org,Johnsonmouth,MS,ryan.org
C00002,Andrea,Sellers,mccoykiara@kelly.com,Stephenfort,WY,kelly.com
C00003,Craig,Hayes,rebeccamiller@yahoo.com,South Stephenshire,LA,yahoo.com
C00004,Bryan,Scott,lawrence05@campbell.info,Chrisland,ND,campbell.info
C00005,Sean,Vasquez,carrie45@yahoo.com,East Dennistown,RI,yahoo.com
C00006,Kevin,Mccarthy,traceyramos@gmail.com,North Matthew,IN,gmail.com
C00007,Amanda,Doyle,scottallen@gmail.com,Joneshaven,VA,gmail.com
C00008,Paul,Campos,sullivanjeremy@horton-adams.com,South Nathanfurt,CT,horton-adams.com
C00009,Mary,Green,dennis03@yahoo.com,Kimberlyview,MD,yahoo.com
C00010,James,Myers,charles58@murillo.net,West Hector,OK,murillo.net


2000

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).orderBy(desc("total_customers")).limit(10).display()

domains,total_customers
gmail.com,374
hotmail.com,360
yahoo.com,331
davis.com,8
brown.com,8
smith.com,7
johnson.com,5
hernandez.com,5
white.com,4
miller.com,4


In [0]:
df_gmail=df.filter(col("domains")=='gmail.com')
df_gmail.limit(10).display()
df_gmail.count()

customer_id,email,city,state,domains,full_name
C00006,traceyramos@gmail.com,North Matthew,IN,gmail.com,Kevin Mccarthy
C00007,scottallen@gmail.com,Joneshaven,VA,gmail.com,Amanda Doyle
C00011,anita65@gmail.com,Houstonfurt,AR,gmail.com,Jacob Le
C00012,beardtravis@gmail.com,East Jenniferview,WI,gmail.com,Chad Banks
C00013,jwood@gmail.com,Lake Gregoryshire,OR,gmail.com,James Martin
C00017,rhondaferguson@gmail.com,West Loriborough,GA,gmail.com,Angela Carter
C00028,steven27@gmail.com,North Gregoryfurt,NM,gmail.com,Barry Baker
C00030,cfuller@gmail.com,Port Coltonton,AL,gmail.com,Holly Collins
C00032,egallegos@gmail.com,Samuelshire,KS,gmail.com,Alexandria Singleton
C00034,stacy78@gmail.com,Gregorybury,IA,gmail.com,Carol Matthews


374

In [0]:
df_hotmail=df.filter(col("domains")=='hotmail.com')
df_hotmail.limit(10).display()
df_hotmail.count()

customer_id,email,city,state,domains,full_name
C00015,daniellowe@hotmail.com,South Courtney,SC,hotmail.com,Diane Harris
C00020,rojassandra@hotmail.com,South Oscar,NV,hotmail.com,Juan Collins
C00031,kramerkaylee@hotmail.com,Travisview,CT,hotmail.com,Jeanette Smith
C00033,lindasummers@hotmail.com,South Andrewchester,CO,hotmail.com,Carrie Wheeler
C00036,jesustaylor@hotmail.com,Port Nicoleborough,AR,hotmail.com,Jacob Mccoy
C00037,eric78@hotmail.com,Millerfurt,VT,hotmail.com,Holly Arnold
C00038,ythomas@hotmail.com,Port Michaelstad,MS,hotmail.com,Robert Haas
C00039,angela55@hotmail.com,Lake Michaelhaven,AL,hotmail.com,Teresa Ward
C00041,pbarnes@hotmail.com,Allisonhaven,AK,hotmail.com,James Brooks
C00043,ronnie19@hotmail.com,South Johnville,OK,hotmail.com,Margaret Sullivan


360

In [0]:
df_yahoo=df.filter(col("domains")=='yahoo.com')
df_yahoo.limit(10).display()
df_yahoo.count()

customer_id,email,city,state,domains,full_name
C00003,rebeccamiller@yahoo.com,South Stephenshire,LA,yahoo.com,Craig Hayes
C00005,carrie45@yahoo.com,East Dennistown,RI,yahoo.com,Sean Vasquez
C00009,dennis03@yahoo.com,Kimberlyview,MD,yahoo.com,Mary Green
C00014,lmoore@yahoo.com,East Tiffanybury,MS,yahoo.com,Thomas Hartman
C00024,andreawilliams@yahoo.com,Stewartport,NM,yahoo.com,Heather Owens
C00025,dawn65@yahoo.com,Port Juliebury,IN,yahoo.com,Christina Bennett
C00026,rebecca20@yahoo.com,Ortizshire,WY,yahoo.com,Mark Harris
C00042,hannasteven@yahoo.com,South Garrettton,IA,yahoo.com,Jenna Lewis
C00048,floresvanessa@yahoo.com,Lake Andrea,MO,yahoo.com,Natalie Gentry
C00051,millerjennifer@yahoo.com,Anthonymouth,CO,yahoo.com,Frank Phillips


331

In [0]:
df=df.withColumn("full_name",concat(col("first_name"),lit(" "),col("last_name")))
df=df.drop("first_name","last_name")
df.limit(10).display()
df.count()

customer_id,email,city,state,domains,full_name
C00001,rushjeff@ryan.org,Johnsonmouth,MS,ryan.org,Emily Mooney
C00002,mccoykiara@kelly.com,Stephenfort,WY,kelly.com,Andrea Sellers
C00003,rebeccamiller@yahoo.com,South Stephenshire,LA,yahoo.com,Craig Hayes
C00004,lawrence05@campbell.info,Chrisland,ND,campbell.info,Bryan Scott
C00005,carrie45@yahoo.com,East Dennistown,RI,yahoo.com,Sean Vasquez
C00006,traceyramos@gmail.com,North Matthew,IN,gmail.com,Kevin Mccarthy
C00007,scottallen@gmail.com,Joneshaven,VA,gmail.com,Amanda Doyle
C00008,sullivanjeremy@horton-adams.com,South Nathanfurt,CT,horton-adams.com,Paul Campos
C00009,dennis03@yahoo.com,Kimberlyview,MD,yahoo.com,Mary Green
C00010,charles58@murillo.net,West Hector,OK,murillo.net,James Myers


2000

### **Data Writing**

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@databricksaniete.dfs.core.windows.net/customers")

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS databricks_cata.silver

In [0]:
%sql
CREATE TABLE if not EXISTS databricks_cata.silver.customers_silver
USING DELTA
LOCATION 'abfss://silver@databricksaniete.dfs.core.windows.net/customers'